In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge import Rouge
import torch
import pandas as pd

In [7]:
model_name = 'declare-lab/flan-alpaca-large'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Downloading: 100%|██████████| 2.50k/2.50k [00:00<00:00, 1.33MB/s]
Downloading: 100%|██████████| 2.42M/2.42M [00:00<00:00, 4.09MB/s]
Downloading: 100%|██████████| 2.20k/2.20k [00:00<00:00, 2.12MB/s]
Downloading: 100%|██████████| 787/787 [00:00<00:00, 527kB/s]
Downloading: 100%|██████████| 3.13G/3.13G [00:44<00:00, 70.9MB/s]


In [8]:
def get_response_and_scores(prompt, reference):
    # Encode & Decode
    inputs = tokenizer.encode("Reply to this prompt: " + prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    input_ids = tokenizer.encode(response, return_tensors='pt')

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss)

    rouge = Rouge()
    scores = rouge.get_scores(response, reference)

    df = pd.DataFrame({
        'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
        'Recall': [scores[0]['rouge-1']['r'], scores[0]['rouge-2']['r'], scores[0]['rouge-l']['r']],
        'Precision': [scores[0]['rouge-1']['p'], scores[0]['rouge-2']['p'], scores[0]['rouge-l']['p']],
        'F1 Score': [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']],
    })

    print(f'ROUGE scores\n{df}')

    return response, perplexity.item(), scores

In [9]:
new_prompt = 'What is the impact of AI?'
reference_text = 'AI has a significant impact on various fields, including healthcare, transportation, and finance. It can automate tasks, improve decision-making, and provide new insights from data.'

response, perplexity, rouge_scores = get_response_and_scores(new_prompt, reference_text)
print('\nResponse:', response)
print('Perplexity:', perplexity)


ROUGE scores
    Metric    Recall  Precision  F1 Score
0  ROUGE-1  0.416667   0.238095  0.303030
1  ROUGE-2  0.166667   0.074074  0.102564
2  ROUGE-L  0.416667   0.238095  0.303030

Response: <pad> AI has had a significant impact on the world, from healthcare to finance. AI has enabled machines to learn and adapt to changing conditions, allowing them to become more efficient and productive. AI has also enabled the development of new technologies, such as autonomous vehicles and autonomous vehicles, which have enabled the development of new products and services. AI has also enabled the development of new technologies, such as autonomous vehicles and autonomous vehicles, which have enabled the development of new products and services.</s>
Perplexity: 1.7496857643127441
